# Model Evaluation Using Cross-Validation for an Advanced Fibrosis Diagnosis Classifier

We learned about the hepatitis C dataset in Activity 3.02, Advanced Fibrosis Diagnosis with Neural Networks of Chapter 3, Deep Learning with Keras. The dataset consists of information for 1385 patients who underwent treatment dosages for hepatitis C. For each patient, 28 different attributes are available, such as age, gender, and BMI, as well as a class label, which can only take two values: 1, indicating advanced fibrosis, and 0, indicating no indication of advanced fibrosis. This is a binary/two-class classification problem with an input dimension equal to 28.

In Chapter 3, Deep Learning with Keras, we built Keras models to perform classification on this dataset. We trained and evaluated the models using training set/test set splitting and reported the test error rate. In this activity, we are going to use what we learned in this topic to train and evaluate a deep learning model using k-fold cross-validation. We will use the model that resulted in the best test error rate from the previous activity. The goal is to compare the cross-validation error rate with the training set/test set approach error rate:

### 1. Import the necessary libraries. Load the dataset from the data subfolder of the Chapter04 folder from GitHub using X = pd.read_csv('../data/HCV_feats.csv'), y = pd.read_csv('../data/HCV_target.csv'). Print the number of examples in the dataset, the number of features available, and the possible values for the class labels.

In [21]:
import pandas as pd 
import numpy as np 


In [22]:
X = pd.read_csv('../data/HCV_feats.csv')
y = pd.read_csv('../data/HCV_target.csv')
print(X.shape)
print(y.shape)

(1385, 28)
(1385, 1)


In [23]:
y.AdvancedFibrosis.value_counts()

1    717
0    668
Name: AdvancedFibrosis, dtype: int64

### 2. Define the function that returns the Keras model. The Keras model will be a deep neural network with two hidden layers, where the first hidden layer is of size 4 and the second hidden layer is of size 2, and use the tanh activation function to perform the classification. Use the following values for the hyperparameters: optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']

In [24]:
from keras.models import Sequential
from keras.layers import Dense
def build_model():
    model = Sequential()

    model.add(Dense(4, input_dim=X.shape[1], activation='tanh'))
    model.add(Dense(2, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer = 'adam', 
        loss = 'binary_crossentropy', 
        metrics = ['accuracy']
    )

    return model

### 3. Build the scikit-learn interface for the Keras model with epochs=100, batch_size=20, and shuffle=False. Define the cross-validation iterator as StratifiedKFold with k=5. Perform k-fold cross-validation on the model and store the scores.

In [25]:
from tensorflow import random

seed = 1

np.random.seed(seed)
random.set_seed(seed)

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [29]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# params = {
#     build_fn: build_model,
#     epochs: 100,
#     batch_size: 20,
#     shuffle: False,
#     verbose: 1
# }

classifier = KerasClassifier(build_fn=build_model, epochs=100, batch_size=20, verbose=1, shuffle=False)

skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=False)

scores = cross_val_score(classifier, X, y, cv=skf)

h 14/100
1108/1108 [==============================] - 0s 115us/step - loss: 0.6763 - accuracy: 0.5632
Epoch 15/100
1108/1108 [==============================] - 0s 119us/step - loss: 0.6748 - accuracy: 0.5650
Epoch 16/100
1108/1108 [==============================] - 0s 114us/step - loss: 0.6736 - accuracy: 0.5632
Epoch 17/100
1108/1108 [==============================] - 0s 156us/step - loss: 0.6724 - accuracy: 0.5641
Epoch 18/100
1108/1108 [==============================] - 0s 138us/step - loss: 0.6714 - accuracy: 0.5668
Epoch 19/100
1108/1108 [==============================] - 0s 131us/step - loss: 0.6703 - accuracy: 0.5713
Epoch 20/100
1108/1108 [==============================] - 0s 140us/step - loss: 0.6694 - accuracy: 0.5767
Epoch 21/100
1108/1108 [==============================] - 0s 130us/step - loss: 0.6685 - accuracy: 0.5740
Epoch 22/100
1108/1108 [==============================] - 0s 133us/step - loss: 0.6676 - accuracy: 0.5722
Epoch 23/100
1108/1108 [==========================

### 4. Print the accuracy for each iteration/fold, plus the overall cross-validation accuracy and its associated standard deviation.

In [30]:
scores

array([0.52346569, 0.51624548, 0.53429604, 0.52346569, 0.58844763])

In [31]:
scores.mean()

0.5371841073036194

In [32]:
scores.std()

0.026272059193352466

The accuracy we received from training set/test set approach we performed in Activity 3.02, Advanced Fibrosis Diagnosis with Neural Networks of Chapter 3, Deep Learning with Keras, was 49.819%, which is lower than the test accuracy we achieved when performing 5-fold cross-validation on the same deep learning model and the same dataset, but lower than the accuracy on one of the folds.

The reason for this difference is that the test error rate resulting from the training set/test set approach was computed by only including a subset of the data points in the model's evaluation. On the other hand, the test error rate here is computed by including all the data points in the evaluation, and therefore this estimation of the model's performance is more accurate and more robust, performing better on the unseen test dataset.

In this activity, we used cross-validation to perform a model evaluation on a problem involving a real dataset. Improving model evaluation is not the only purpose of using cross-validation, and it can be used to select the best model or parameters for a given problem as well.